In [2]:
# pip install pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import re
import time

In [2]:
# Set Spark Configuration
conf = SparkConf() \
    .setMaster("local[*]") \
    .setAppName("WordFrequency") \
    .setExecutorEnv("spark.executor.memory", "4g") \
    .setExecutorEnv("spark.driver.memory", "4g")


# Create a SparkSession
spark = SparkSession.builder \
    .config(conf = conf) \
    .getOrCreate()

23/05/24 22:49:14 WARN Utils: Your hostname, Tanmays-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.123 instead (on interface en0)
23/05/24 22:49:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/24 22:49:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
start_time = time.time()
stopwords = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 'should',
 'now']

# Load the dataset into an RDD (Resilient Distributed Dataset)
data_rdd = spark.sparkContext.textFile("/Users/tanmaysingla/Downloads/dataset_updated/data_16GB.txt")

# Split each line of the RDD into individual words and convert them to lowercase
words_rdd = data_rdd.flatMap(lambda line: line.lower().split(" "))

# Clean the words RDD
clean_words_rdd = words_rdd.map(lambda word: re.sub(r'[^a-zA-Z0-9]', '', word)).filter(lambda x: x != '' and x not in stopwords) # and len(x) > 6

# Map each word to a count of 1
word_counts = clean_words_rdd \
    .map(lambda word: (word, 1))

# Reduce by key to get the count of each word
word_counts = word_counts \
    .reduceByKey(lambda a, b: a + b)

# Sort the word counts in descending order
sorted_word_counts = word_counts \
    .sortBy(lambda x: x[1], ascending=False)

# Take the top 100 words
top_100_words = sorted_word_counts \
    .take(100)

algo_time = time.time() - start_time
print(top_100_words)
print(algo_time)

[('said', 16983022), ('would', 5829046), ('one', 5827766), ('new', 5619230), ('also', 4618215), ('us', 4602714), ('people', 4302072), ('last', 4096901), ('year', 4011799), ('two', 3964133), ('first', 3839806), ('mr', 3733886), ('years', 3637637), ('time', 3635749), ('could', 3374876), ('like', 3058687), ('government', 2520220), ('says', 2417349), ('may', 2393102), ('get', 2346948), ('many', 2344537), ('back', 2295703), ('million', 2254837), ('three', 2250192), ('president', 2188344), ('even', 2175397), ('made', 2130240), ('make', 2104941), ('told', 2092901), ('since', 2041702), ('world', 2039950), ('percent', 2022484), ('police', 2014247), ('well', 1965769), ('still', 1956830), ('state', 1948082), ('way', 1936599), ('much', 1878810), ('say', 1866766), ('day', 1831550), ('week', 1829328), ('home', 1827480), ('take', 1785403), ('per', 1779036), ('work', 1770892), ('going', 1744157), ('think', 1670553), ('company', 1665814), ('good', 1635403), ('dont', 1629480), ('next', 1605312), ('inclu

In [4]:
# data_rdd = spark.sparkContext.textFile("hdfs://localhost:9870/data/input/small_50MB_dataset.txt").repartition(8)

In [ ]:
# For all words in 2.5GB: 239.72171878814697
# For words with len > 6 in 2.5GB: 212.92708086967468

In [ ]:
# For all words in 16GB: 1407.3491878509521
# For words with len > 6 in 16GB: 1284.740023612976